<h1> Summarize Customer Reviews Using Amazon Bedrock </h1>
<br>

<p> Before starting, please make sure this notebook is using <b>conda_python3</b> kernel from the top right! </p>

<p> In this notebook, let's learn how to use Amazon Bedrock to summarize all the customer reviews of a product </p>

<p> To run this notebook, go to Cell -> Run All. Inspect the output of each cell block. </p>

<b> Please read the following instructions carefully! </b>
    <ul>
    <li>We highly recommend you to run all cells and inspect output rather than running the cells individually to save time as well as avoid any issues.  
    <li>This notebook is for your understanding. Running this notebook is NOT required for proceeding with the next steps of your workshop.
    <li>In case your notebook does not run as expected or if you run into any errors, please proceed with the next steps provided in the Workshop instructions. 
    <li>If you choose to run the notebooks, please read the comments in the markdown and inspect the output of each cell.
    </ul>

<h3> Install required dependencies </h3>
<p> <b>Note:</b> If you notice any ERRORs from the following cell, ignore them and proceed with the next cells.</p><br>

In [ ]:
%pip install --quiet --no-build-isolation --upgrade \
    "boto3==1.28.63" \
    "awscli==1.29.63" \
    "botocore==1.31.63" \
    "langchain==0.0.309"

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

<h3> Initialize Bedrock client </h3><br>

In [ ]:
 boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

<h3> Initialize LLM </h3>
<p> Using Langchain, initialize an LLM instance for text summarization. </p>

In [ ]:
textsumm_llm = Bedrock(
                model_id="anthropic.claude-instant-v1",
                client=boto3_bedrock)

<p> Create a prompt template with the variables: product name and consolidated list of customer reviews for the product. </p> 

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["product_name","reviews"],
    template="""
    
    Human: Provide a review summary including pros and cons based on the customer reviews for the product {product_name}. This summary will be updated in the product webpage. Customer reviews are enclosed in <customer_reviews> tag. 
    
        <customer_reviews>
            {reviews}
        <customer_reviews>
        
    Assistant:
    
    """)

<h3> Provide customer reviews data for the prompt </h3>

<p> This data will be used to construct our prompt which will be passed to the LLM to customize customer reviews. </p> 

In [ ]:
product_name="Treadlite Shoe"

# There are 4 customer reviews for this product 

product_reviews = ["""I've owned these shoes for about 6 months now and have put over 300 miles on them. They are super lightweight and provide excellent cushioning and support for long runs. The breathable mesh keeps my feet from overheating even on hot summer days. The only minor issue is that the tread is starting to show some light wear but for a $200 shoe I'm really impressed with the durability. Overall these are a fantastic value for any serious runner.""",
                   """While the Treadlite shoes look sleek and are very lightweight, I'm finding they don't provide enough support for high intensity workouts. On long runs or easy miles they feel fine but anytime I do speedwork or hill repeats my feet and ankles feel beat up afterwards. The cushioning also seems to flatten out quickly. I'm only a few months into a training plan and they are barely holding up. For the price I expected them to last longer. I like the brand but may look elsewhere for my next pair of shoes.""",
                   """I bought these shoes to use for light gym workouts and occasional runs but after a few weeks I realized they weren't supportive enough for any intense exercise. The upper material is very thin and offers little protection or structure for high-impact activities. On my long run last weekend my feet and ankles were sore afterwards. They look and feel lightweight but lack sturdiness. Fine for walking around casually but I wouldn't recommend them for serious athletes or those training for races and would look for a shoe with better cushioning and stability.""",
                   """Treadlite shoes promise lightweight comfort, but they failed to deliver for me. Within a few weeks of regular use, the thin material started wearing down already. There are holes forming on the sides and small tears along the seams. The fabric just does not feel durable enough. Additionally, I found these shoes to be poorly cushioned. After a 5 mile run, my feet and knees were sore from the lack of support and bounce. It was like running directly on the hard pavement. The shoe provides almost no impact absorption for a runner. The sizing is also off. I bought my normal size but the shoes feel restrictive, like my feet are being squeezed. They did not stretch or mold to the shape of my foot over time like other running shoes. It's an uncomfortable fit that leaves my feet feeling constricted after runs. Between the lack of cushioning, poor durability of materials, and sizing issues, these Treadlite shoes have been a big disappointment. For a brand focused on running, the design flaws mean they are not well-suited for the needs of active individuals. I cannot recommend these shoes and would not purchase from this brand again based on my experience. Runners deserve better quality and performance than what Treadlite provided."""]

# Loading this list in HTML tag format to pass as a prompt to the LLM. 
# Doing it this way helps LLM understand our instruction better

review_digest = ''

for review in product_reviews:
    review_digest += "<review>" + '\n'
    review_digest += review + '\n'
    review_digest += "</review>" + '\n\n'
        
# print_ww(review_digest)

<p> Let's construct the prompt by passing all the variables to the prompt template. </p>

In [ ]:
prompt = prompt_template.format(product_name=product_name,
                                reviews=review_digest)

Let's see how the prompt looks

In [ ]:
print_ww(prompt)

In [ ]:
response = textsumm_llm(prompt)
print_ww(response)

<h4> We can also try to generate the customer review summary using another LLM. Let's use Amazon Titan LLM from Bedrock </h4>

In [ ]:
textsumm_llm = Bedrock(
                model_id="amazon.titan-tg1-large",
                client=boto3_bedrock)

In [ ]:
response = textsumm_llm(prompt)
print_ww(response)

<h3> You've successfully created response to the customer reviews for a product with Amazon Bedrock!</h3>

<p> Please stop the notebook kernel before proceeding. </p>

<h4> Now, let's learn how to integrate Amazon Bedrock and Langchain into your web application to do the same. Please go back to Workshop Studio and follow the instructions to replicate this code into your Cloud9 environment. </h4>